In [1]:
# number of scenarios, here is 2
S = 2

# number of poential charging staion locations
I = 4

# number of cars 
H = 5

# 4 time intervals within an hour 
T = 4

# Fixed cost of each charging station
station_cost = 1

# Purchasing cost of each car
car_cost = 1

# Income of each accepted trip
income_per_car = 20

# Capacity of each charging station, i.e. number of charging slots can be built at each charging station
capacity = 10

In [2]:
import pandas as pd
# Define the nodes in the graph
root = 'root'
sink = 'sink'

V = [(s, i, t) for s in range(S) for i in range(I) for t in range(T)] + [sink] 

# Waiting arcs
waiting_arcs = [(s, (i, t), (i, t+1)) for s in range(S) for i in range(I) for t in range(T)]

# Travel arcs
path1 = r"C:\Users\yanzh\Desktop\code_and_data\5. Model\sample_trips_data_1.csv"
df1 = pd.read_csv(path1)
path2 = r"C:\Users\yanzh\Desktop\code_and_data\5. Model\sample_trips_data_2.csv"
df2 = pd.read_csv(path2)
dfs = {0: df1, 1: df2}
travel_arcs = []
for s, data in dfs.items():
    for k in range(len(data)):
        row = data.iloc[k]
        starting_point = row["starting_point"]
        starting_date = row["starting_date"]
        ending_point = row["ending_point"]
        ending_date = row["ending_date"]
        arcs = (s, k, (starting_point, starting_date), (ending_point, ending_date))
        travel_arcs.append(arcs)
        
# Final collection arcs
final_collection_arcs = [(s, (i, T), sink) for s in range(S) for i in range(I)]

In [3]:
import gurobipy as gp
from gurobipy import Model, GRB, quicksum
import pandas as pd 

# Initialize the model
m = Model('CSLP')


# First stage decision variable
y_i = m.addVars(range(4), vtype=GRB.BINARY, name='build_variable')
L_i = m.addVars(range(4), vtype=GRB.INTEGER, name='purchased_car', lb=0, ub=5)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-08


In [4]:
# Second stage decision variale
x_k = m.addVars([(s, k) for s in range(S) for k in range(len(dfs[s]))], vtype=GRB.BINARY, name='accpted_trip')

In [5]:
# Initialize f_ha as a dictionary
f_ha = {}

# Process waiting_arcs and final_collection_arcs
for arc in waiting_arcs + final_collection_arcs:
    s, src, dst = arc  # These are tuples (s, src, dst)
    for h in range(H):
        f_ha[(s, h, arc)] = m.addVar(vtype=GRB.BINARY, name=f'flow_realized_by_car_{s}_{h}_{src}_{dst}')

# Process travel_arcs
for arc in travel_arcs:
    s, k, src, dst = arc  # These are quadruples (s, k, src, dst)
    for h in range(H):
        f_ha[(s, h, arc)] = m.addVar(vtype=GRB.BINARY, name=f'flow_realized_by_car_{s}_{h}_{src}_{dst}')


In [6]:
x_hk = m.addVars([(s, k, h) for s in range(S) for h in range(H) for k in range(len(dfs[s]))], vtype=GRB.BINARY, name='trip_realized_by_car')

In [7]:
# Precompute incoming arcs for each (s, i, t)
incoming_arcs_dict = { (s, i, t): [] for s in range(S) for i in range(I) for t in range(T + 1) }
outgoing_arcs_dict = { (s, i, t): [] for s in range(S) for i in range(I) for t in range(T) }

for arc in waiting_arcs:
    s, src, dst = arc
    if (s, dst[0], dst[1]) in incoming_arcs_dict:
        incoming_arcs_dict[(s, dst[0], dst[1])].append(arc)
    if (s, src[0], src[1]) in outgoing_arcs_dict:   
        outgoing_arcs_dict[(s, src[0], src[1])].append(arc)

for arc in travel_arcs:
    s, k, src, dst = arc
    if (s, dst[0], dst[1]) in incoming_arcs_dict:
        incoming_arcs_dict[(s, dst[0], dst[1])].append(arc)
    if (s, src[0], src[1]) in outgoing_arcs_dict:
        outgoing_arcs_dict[(s, src[0], src[1])].append(arc)

$$
\text{max} \quad \sum_{s \in S} p_s \sum_{k \in K^s} i_k x_k - \sum_{i \in I} f_i y_i - c \sum_{i \in I} L_i
$$


In [8]:
total_demand_level = [0.5, 0.5]
# Objective function: maimize profit
m.setObjective(
    quicksum(
        total_demand_level[s] * quicksum(income_per_car * x_k[s, k] for k in range(len(dfs[s])))
        for s in range(S)
    ) -
    quicksum(station_cost * y_i[i] for i in range(I)) -
    car_cost * quicksum(L_i[i] for i in range(I)),
    GRB.MAXIMIZE
)

#### Constraint 3.2
$$\sum_{i\in I}{f_i y_i + c\sum_{i\in I} L_i} \leq W$$

This is the budget constraint, $W$ is the limited budget for all costs.

In [9]:
# Budget constraints
budget = 1000
m.addConstr(
    station_cost * quicksum(y_i[i] for i in range(I)) + 
    car_cost * quicksum(L_i[i] for i in range(I)) <= budget,
    name='budget_constraint'
)

<gurobi.Constr *Awaiting Model Update*>

#### Constraint 3.3

$$
\sum_{i \in I} L_i \leq N \tag{1} 
$$


In [10]:
# # Total number of cars should be fixed as 5 cars
m.addConstr(quicksum(L_i[i] for i in range(I)) <= H, name='total_number_of_cars')


<gurobi.Constr *Awaiting Model Update*>

### Constraints 3.4 Battery Limitation

### Constraints 3.5 initial allocation
$$
\sum_{h=1}^H \sum_{a \in \delta^{+}\left(i_0\right)} f_a^h = L_i y_i \qquad \quad \forall s \in S, \quad \forall i_0 \in V_0^s 
$$

Constraints \ref{equ:initial allocation} impose restrictions on nodes $i_0$, which represents node $i$ of TLG in the initial state ($t=0$), and $V_0^s$ represents the nodes at the initial time $t=0$ within the scenario $s$. Specifically, the sum of all arcs originating from built charging station $i$, including both waiting arcs and traveling arcs, must equal the initial number of cars at charging station $i$. These constraints can guarantee that each purchased car is first allocated to its corresponding built charging station.

In [11]:
# Ensure that the initial number of cars at each station equals the outgoing flow at t=0
for i in range(I):
    for s in range(S):
        outgoing_arcs = outgoing_arcs_dict[(s, i, 0)]
        if outgoing_arcs:
            m.addConstr(
                quicksum(f_ha[(s, h, arc)] for h in range(H) for arc in outgoing_arcs if (s, h, arc) in f_ha) == L_i[i] * y_i[i],
                name=f"initial_flow_station_{i}_scenario_{s}"
            )
        else:
            print(f"No outgoing arcs for station {i}, scenario {s} at t=0")

### Constraints 3.6

$$\sum_{h=1}^H x_k^h = x_k, \qquad\forall s \in S, k \in K^s $$

It ensures that exactly one car is assigned to each accepted trip.

In [12]:
# Constraint: one car per accepted trip
for s in range(S):
    for k in range(len(dfs[s])):
        m.addConstr(
            quicksum(x_hk[s, k, h] for h in range(H)) == x_k[s, k], 
            name=f"one_car_per_accepted_trip_s{s}_k{k}"
        )

### Constraints 3.7

$$
\sum_{h \in H} \sum_{k \in K^s : o_k = i, s_k = 0} x_k^h \leq L_i y_i, \qquad \forall i \in I, \quad \forall s \in S
$$

In [13]:
# Add the constraint to ensure that the number of trips assigned to each station at t=0 does not exceed the number of cars at that station.
for i in range(I):
    for s in range(S):
        # Create a constraint for each station and scenario at t=0
        m.addConstr(
            quicksum(x_hk[s, k, h] for h in range(H) for k in range(len(dfs[s])) 
                     if dfs[s].iloc[k]['starting_point'] == i and dfs[s].iloc[k]['starting_date'] == 0) <= L_i[i] * y_i[i],
            name=f'trip_assignment_limit_station_{i}_scenario_{s}'
        )

### Constrain 3.8

$$\sum_{h=1}^H \sum_{a \in \delta^{+}\left(i_t\right) \cap\left(A_W^s \cup A_C^s\right)} f_a^h \leq C_i y_i \qquad \forall s \in S, \quad\forall i_t \in V^s \backslash\left\{r^s, s^s\right\}$$

It ensures that the quantity of vehicles concurrently parked at station $i$ does not surpass the available number of charging slots at said station.
Observe that final collection arcs need to be considered on the left-hand side to ensure that the capacity constraints are also met at the end of the planning period.

In [14]:
# Capacity constraints
for s in range(S):
    for i in range(I):
        # Directly handling the final collection arcs to 'sink'
        final_arc_key = (s, (i, T), 'sink')
        m.addConstr(
            quicksum(f_ha[(s, h, final_arc_key)] for h in range(H) if (s, h, final_arc_key) in f_ha) <= capacity * y_i[i], 
            name=f'capacity_collection_arcs_s{s}_i{i}'
        )

        for t in range(T):
           # Filter and append arcs relevant to the current (s, i, t) 
           outgoing_waiting_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and src == (i, t)]

           # Filter arcs that are in f_ha
           valid_outgoing_arcs = [arc for arc in outgoing_waiting_arcs if any((s, h, arc) in f_ha for h in range(H))]
           
           # Add constraint
           m.addConstr(quicksum(f_ha[(s, h, arc)] for h in range(H) for arc in valid_outgoing_arcs) <= capacity * y_i[i], name=f'capacity_waiting_arcs_s{s}_i{i}_t{t}')      

### Constraints 3.9

$$    f^h[\delta^{-}\left(i_t\right)] \leq y_i \quad \forall h \in\{1,2, \ldots, H\}, \quad \forall s \in S, \quad \forall i_t \in V^s \backslash\left\{r^s, s^s\right\}$$

It ensures that car can only enter the built station. It includes waiting arcs (cars only car wait at the built station), and traveling arcs (cars can only park at the built station)

In [15]:
for s in range(S):
    for i in range(I):
        for t in range(1, T+1):
            # Given s, i, t, a specific waiting arc can be identified
            incoming_arcs = incoming_arcs_dict[(s, i, t)]

            for h in range(H): # Ensure the constraint is applied for each car h
                # Add the combined constraint
                if incoming_arcs:
                    m.addConstr(
                        quicksum(f_ha[(s, h, arc)] for arc in incoming_arcs if (s, h, arc) in f_ha) <= y_i[i],
                        name=f'only_enter_built_station_s{s}_i{i}_t{t}_h{h}'
                    )

### Constraint 3.10

$$f^h\left[\delta^{-}\left(i_t\right)\right]=f^h\left[\delta^{+}\left(i_t\right)\right] \quad \forall h \in\{1,2, \ldots, H\}, \quad\forall s \in S, \quad\forall i_t \in V^s \backslash\left\{r^s, s^s\right\}$$

Flow conservation ensures that the route of each car must correspond to a path through the time-expanded location graph for each scenario

In [16]:
# Flow conservation constraint
for s in range(S):
    for i in range(I):
        for t in range(T):    
            if (i, t) == (i, 0):
                continue # Skip starting point

            incoming_arcs = incoming_arcs_dict[(s, i, t)]
            outgoing_arcs = outgoing_arcs_dict[(s, i, t)]

            for h in range(H):
                m.addConstr(
                    quicksum(f_ha[(s, h, arc)] for arc in incoming_arcs if (s, h, arc) in f_ha) ==
                    quicksum(f_ha[(s, h, arc)] for arc in outgoing_arcs if (s, h, arc) in f_ha),
                    name=f"flow_conservation_s{s}_i{i}_t{t}_h{h}"
                )

### Constraint 3.11

$$\sum_{a \in A_{T}^s(k)} f_a^h=x_k^h \quad \forall h \in\{1,2, \ldots, H\}, \quad \forall s \in S, \quad \forall k \in K^s$$

This equation illustrates all action of one car.

In [17]:
# Precompute travel arcs for each scenario and trip
travel_arcs_per_scenario_trip = {
    (s, k): [arc for arc in travel_arcs if arc[0] == s and arc[1] == k]
    for s in range(S)
    for k in range(len(dfs[s]))
}

# Add constraints to ensure all actions of one car
for s in range(S):
    for k in range(len(dfs[s])):
        relevant_arcs = travel_arcs_per_scenario_trip[(s, k)]
        for h in range(H):
            m.addConstr(
                quicksum(f_ha[(s, h, arc)] for arc in relevant_arcs if (s, h, arc) in f_ha) == x_hk[s, k, h],
                name=f'all_action_one_car_s{s}_k{k}_h{h}'
            )

### Constraint 3.12

$$ f_a^h \leq f_{a^{\prime}}^h \\

\forall h \in\{1,2, \ldots, H\}, \forall s \in S, \forall k \in K^s, \\
\forall a=\left(i_{s_k}, j_{e_k}\right) \in A_{\mathrm{T}}^s(k), \\
\forall a^{\prime}=\left(j_t, j_{t^{\prime}}\right) \in A_{W}^s, \\
t = e_k, t^{\prime} = e_k+\left\lceil\frac{b_k}{\rho}\right\rceil $$

This equation force each car must fully charge the battery after completing the service.

In [18]:
# # 假定完全充电时间为15分钟
# fully_charging_time = 1

# # 遍历所有场景
# for s in range(S):
#     # 从 dfs 中获取场景 s 的需求列表
#     for k in range(len(dfs[s])):  # 确保使用 len(dfs[s]) 以处理每个场景的具体需求数量
#         # 获取场景 s 的所有旅行弧
#         AT_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s]
#         if AT_arcs:
#             for h in range(H):
#                 # 获取特定需求 k 的旅行弧
#                 at_arc = AT_arcs[k]
#                 dp = at_arc[3][0]  # 目的地点
#                 et = at_arc[3][1]  # 结束时间

#                 t = et
#                 t_prime = t + fully_charging_time  # 计算需要的充电结束时间

#                 # 查找对应的等待弧，时间和地点都要匹配
#                 AW_arcs = [(s_arc, src, dst) for s_arc, src, dst in waiting_arcs if s_arc == s and src[1] == t and src[0] == dp]

#                 # 为每一个符合条件的等待弧添加约束
#                 for aw_arc in AW_arcs:
#                     travel_key = (s, h, at_arc)
#                     waiting_key = (s, h, aw_arc)
#                     if travel_key in f_ha and waiting_key in f_ha:
#                         m.addConstr(f_ha[travel_key] <= f_ha[waiting_key], name=f"charging_constraint_s{s}_k{k}_h{h}_arct{at_arc}_arcw{aw_arc}")
#                     else:
#                         print(f"Keys not found in f_ha for travel arc {travel_key} or waiting arc {waiting_key}")


In [23]:
fully_charging_time = 1  # 充电时间为15分钟

for s in range(S):
    for k in range(len(dfs[s])):  # 处理每个场景的需求量
        # 获取场景 s 的所有旅行弧
        AT_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s]
        
        if AT_arcs:
            for h in range(H):
                # 获取特定需求 k 的旅行弧
                at_arc = AT_arcs[k]
                dp = at_arc[3][0]  # 目的地点
                et = at_arc[3][1]  # 结束时间

                # 为每一个旅行弧添加连续的等待时间弧，直到完全充电时间或达到总时间 T
                for minute in range(fully_charging_time):
                    t_start = et + minute
                    t_end = t_start + 1

                    # 如果 t_end 达到或超过 T，则停止添加该旅行弧的更多等待时间弧
                    if t_end >= T:
                        break

                    AW_arc = (s, (dp, t_start), (dp, t_end))  # 定义等待弧

                    # 检查是否每个等待弧都在 f_ha 中
                    if (s, h, at_arc) in f_ha and (s, h, AW_arc) in f_ha:
                        # 为每个旅行弧和对应的等待弧添加流量约束
                        m.addConstr(f_ha[(s, h, at_arc)] <= f_ha[(s, h, AW_arc)], 
                                    name=f"charging_constraint_s{s}_k{k}_h{h}_minute{minute}_arct{at_arc}_arcw{AW_arc}")
                    else:
                        # 打印出缺失的弧信息，以便进一步调试
                        if (s, h, at_arc) not in f_ha:
                            print(f"Travel arc not found in f_ha: {(s, h, at_arc)}")
                        if (s, h, AW_arc) not in f_ha:
                            print(f"Waiting arc not found in f_ha: {(s, h, AW_arc)}")


In [ ]:
# 假定完全充电时间为15分钟，每分钟一个等待弧
fully_charging_time = 15

for s in range(S):
    # 获取场景 s 的需求列表
    for k in range(len(dfs[s])):
        # 获取场景 s 的所有旅行弧
        AT_arcs = [(s_arc, k, src, dst) for s_arc, k, src, dst in travel_arcs if s_arc == s]
        if AT_arcs:
            for h in range(H):
                # 获取特定需求 k 的旅行弧
                at_arc = AT_arcs[k]
                dp = at_arc[3][0]  # 目的地点
                et = at_arc[3][1]  # 结束时间

                # 计算从 et 开始的 15 分钟的等待时间弧
                for minute in range(fully_charging_time):
                    t_start = et + minute  # 每个等待弧的开始时间
                    t_end = t_start + 1  # 每个等待弧的结束时间（1 分钟后）

                    # 创建一个等待弧
                    AW_arc = (s, (dp, t_start), (dp, t_end))

                    # 检查是否有相应的键
                    travel_key = (s, h, at_arc)
                    waiting_key = (s, h, AW_arc)

                    if travel_key in f_ha and waiting_key in f_ha:
                        # 添加约束：每一个旅行弧的流量 <= 对应的等待弧的流量
                        m.addConstr(f_ha[travel_key] <= f_ha[waiting_key], 
                                    name=f"charging_constraint_s{s}_k{k}_h{h}_minute{minute}_arct{at_arc}_arcw{AW_arc}")
                    else:
                        # 输出详细的调试信息
                        print(f"Travel arc not found: {travel_key} in f_ha: {travel_key in f_ha}")
                        print(f"Waiting arc not found: {waiting_key} in f_ha: {waiting_key in f_ha}")


In [20]:
# Before adding constraints
initial_constr_count = m.NumConstrs

In [21]:
m.update()

In [ ]:
# After adding constraints
final_constr_count = m.NumConstrs
print(f'Number of constraints added: {final_constr_count - initial_constr_count}')

In [ ]:
for constr in m.getConstrs():
    print(f"{constr.ConstrName}: {m.getRow(constr)} {constr.Sense} {constr.RHS}")

In [ ]:
# Solve the model
m.optimize()

# Check optimization status
if m.Status == GRB.OPTIMAL:
    print("Optimization was successful. Printing results.")
    # Initialize total_income
    total_income = 0

    for s in range(S):
        income_per_scenario = 0
        for k in range(len(dfs[s])):
            x_value = x_k[s, k].X  # Get the decision variable value for scenario s and car type k
            income_contribution = income_per_car * x_value
            income_per_scenario += income_contribution
            print(f"x_k[{s}, {k}].X = {x_value}, contributes {income_contribution} to income")

        # Multiply by total demand level for the scenario
        print(f"Total income contribution (before scaling by demand) from scenario {s}: {income_per_scenario}")
        income_per_scenario *= total_demand_level[s]
        total_income += income_per_scenario
        print(f"Total income contribution from scenario {s}: {income_per_scenario}")

    total_station_cost = sum(station_cost * y_i[i].X for i in range(I))
    total_car_cost = car_cost * sum(L_i[i].X for i in range(I))
    
    print(f"Total income: {total_income}")
    print(f"Total station cost: {total_station_cost}")
    print(f"Total car cost: {total_car_cost}")
    
    objective_value = total_income - total_station_cost - total_car_cost
    print(f"Objective value (calculated): {objective_value}")
    print(f"Objective value (from solver): {m.ObjVal}")
    
    for i in range(I):
        if y_i[i].X > 0.5:
            print(f"Build a charging station at location {i} with {L_i[i].X} cars.")

    # Collect and print all f_ha variables with values greater than 0.5
    f_ha_results = []
    for (s, h, arc), var in f_ha.items():
        if var.X > 0.5:
            result = [s, h, arc]
            f_ha_results.append(result)

    # Collect and print all x_k variables with values greater than 0.5
    x_k_results = []
    for (s, k), var in x_k.items():
        if var.X > 0.5:
            result = [s, k]
            x_k_results.append(result)

    # Collect and print all x_hk variables with values greater than 0.5
    x_hk_results = []
    for (s, k, h), var in x_hk.items():
        if var.X > 0.5:
            result = [s, k, h]
            x_hk_results.append(result)
else:
    print(f"Optimization issue with status code: {m.Status}")


In [ ]:
# Profit
print(f'Profit is {total_income}')
# Return on Investment (ROI)
ROI = round(total_income / (total_station_cost + total_car_cost), 4)
print(f'Return on Investment (ROI) is {ROI}')

# Demand Satisfaction Ratio (DSR)
dsr = 0
for s in range(S):
    count = len([item for item in x_k_results if item[0] == s])
    dsr += (count / len(dfs[s])) * total_demand_level[s]
    dsr = round(dsr, 4)
print(f'Demand Satisfaction Ratio (DSR) is {dsr}')


# Charging Station Utilization Rate (CSU)
CSU_total = 0

for s in range(S):  # Iterate over each scenario
    scenario_utilization = 0  # Initialize the total utilization for the scenario

    for i in range(I):  # Iterate over each station
        avg_utilization_i = 0  # Initialize average utilization for station i
        
        for t in range(T):  # Iterate over each time period t
            # Filter f_ha_results to find arcs associated with station i at time t in scenario s
            result_fha = [
                item for item in f_ha_results 
                if item[0] == s and len(item[2]) == 3 
                and item[2][1][0] == i and item[2][1][1] == t
            ]
            
            # Calculate the utilization rate at time t for station i
            utilization_rate_t = sum(1 for _ in result_fha) / capacity  # Number of cars at station i at time t divided by capacity
            
            # Accumulate utilization for station i
            avg_utilization_i += utilization_rate_t
        
        # Compute the average utilization for station i over all time periods T
        avg_utilization_i /= T
        
        # Accumulate the utilization for the scenario
        scenario_utilization += avg_utilization_i
    
    # Multiply the scenario utilization by its probability p_s and add to the total CSU
    CSU_total += total_demand_level[s] * scenario_utilization
CSU_total = round(CSU_total, 4)

# Step 2: Print or return the final CSU (Charging Station Utilization)
print(f"Charging Station Utilization (CSU): {CSU_total}")